In [1]:
import requests
import datetime
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

In [2]:
# Finnhub API key (replace with your own if needed)
API_KEY = 'cv1bm91r01qhkk81os0gcv1bm91r01qhkk81os10'

def fetch_finnhub_news(symbol="AVGO", days=30):
    """
    Fetch company news for the given symbol from Finnhub over the past `days` days.
    Parameters:
      - symbol: Stock ticker symbol (default "AVGO" for Broadcom)
      - days: Number of days in the past to fetch news (default is 30)
    Returns:
      A list of news articles (each is a dictionary with keys like 'headline', 'datetime', etc.)
    """
    today = datetime.date.today()
    start_date = (today - datetime.timedelta(days=days)).strftime("%Y-%m-%d")
    end_date = today.strftime("%Y-%m-%d")
    
    url = f"https://finnhub.io/api/v1/company-news?symbol={symbol}&from={start_date}&to={end_date}&token={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        news_data = response.json()
        return news_data
    else:
        print(f"Error fetching news: {response.status_code} - {response.text}")
        return []


In [3]:
# -----------------------------
# FinBERT Sentiment Analysis
# -----------------------------

# Load FinBERT model and tokenizer from HuggingFace
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
finbert_model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
finbert_model.eval()

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [4]:
def get_sentiment_score(text):
    """
    Uses FinBERT to compute a sentiment score for the given text.
    Score = (Positive probability - Negative probability), range ~[-1, 1].
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = finbert_model(**inputs)
    scores = torch.softmax(outputs.logits, dim=1)
    # Assuming label order: 0 = negative, 1 = neutral, 2 = positive
    pos_prob = scores[0][2].item()
    neg_prob = scores[0][0].item()
    sentiment = pos_prob - neg_prob
    return sentiment

In [5]:
def compute_weighted_sentiment(news_articles, decay_rate=0.1):
    """
    Computes a weighted aggregated sentiment score from a list of news articles.
    
    Parameters:
      news_articles: List of dictionaries, each with 'datetime' (Unix timestamp)
                     and 'headline' for sentiment analysis.
      decay_rate: Controls how fast the weight decays with age (default 0.1).
    
    Returns:
      aggregated_sentiment: Weighted average sentiment score (float).
    """
    today = datetime.datetime.now()
    weighted_scores = []
    total_weight = 0.0
    
    for article in news_articles:
        text = article.get('headline', '')
        if not text:
            continue
        
        sentiment = get_sentiment_score(text)
        article_date = datetime.datetime.fromtimestamp(article['datetime'])
        age_days = (today - article_date).days
        weight = np.exp(-decay_rate * age_days)
        
        weighted_scores.append(sentiment * weight)
        total_weight += weight
    
    if total_weight > 0:
        aggregated_sentiment = sum(weighted_scores) / total_weight
    else:
        aggregated_sentiment = 0.0
        
    return aggregated_sentiment


In [6]:
# -----------------------------
# Main: Retrieve News and Compute Sentiment
# -----------------------------

if __name__ == '__main__':
    # Retrieve news articles for Broadcom (AVGO) for the past 30 days
    news_articles = fetch_finnhub_news("AVGO", days=30)
    print(f"Retrieved {len(news_articles)} articles.")
    
    # Print out details for each article (optional)
    for article in news_articles:
        article_date = datetime.datetime.fromtimestamp(article['datetime']).strftime('%Y-%m-%d %H:%M:%S')
        print("Date:", article_date)
        print("Headline:", article['headline'])
        print("Source:", article.get('source', 'N/A'))
        print("-" * 40)
    
    # Compute and print the aggregated weighted sentiment score
    aggregated_sentiment = compute_weighted_sentiment(news_articles, decay_rate=0.1)
    print(f"\nWeighted Aggregated Sentiment Score: {aggregated_sentiment:.3f}")

Retrieved 194 articles.
Date: 2025-03-01 17:31:01
Headline: Broadcom: Margin Of Safety Still Low Despite Price Pullback
Source: SeekingAlpha
----------------------------------------
Date: 2025-03-01 16:00:00
Headline: 5 Relatively Secure And Cheap Dividend Stocks, Yields Upto 8% (March 2025)
Source: SeekingAlpha
----------------------------------------
Date: 2025-03-01 08:57:43
Headline: Broadcom: Must Hold Until Correction Stops
Source: SeekingAlpha
----------------------------------------
Date: 2025-02-28 21:39:00
Headline: Chip and Software Stocks Are Below Key Levels. How Low They Could Go.
Source: MarketWatch
----------------------------------------
Date: 2025-02-28 21:00:00
Headline: Nvidia's Solid Earnings, Jobs Expectations And Fighting Inflation
Source: SeekingAlpha
----------------------------------------
Date: 2025-02-28 19:10:06
Headline: Nvidia Could Face 'Significant' New China AI Trade Restrictions
Source: DowJones
----------------------------------------
Date: 2025-02-2